In [1]:
import numpy as np
import pandas as pd

import os,ast,shutil,copy

from bokeh.plotting import figure, gridplot 
from bokeh.io import output_file, show, output_notebook
output_notebook()


def bokeh_show(
        params,
        df_cross,
        show_figures1, 
        show_figures2, wps_fig2,
        color_cross):

    colors = [subm['color'] for subm in params['subm']]
    
    def dossier(js,subms,cols):
        def quant(i,js,subms,cols):
            return {"c" : i, "q" : sum([1 for subm in cols[i] if subm == subms[js]])}
        return {
            'name' : subms[js],
            'q_in' : [quant(i,js,subms,cols) for i in range(len(subms))]
        }
    alls = pd.read_csv(f'tida_desc.csv')
    matrix = [ast.literal_eval(str(row.alls)) for row in alls.itertuples()]
    subms = sorted(matrix[0])
    cols = [[data[i] for data in matrix] for i in range(len(subms))]
    df_subms = pd.DataFrame({f'col_{i}': [x[i] for x in matrix] for i in range(len(subms))})
    dossiers = [dossier(js,subms,cols) for js in range(len(subms))]
    subm_names = [one_dossier['name'] for one_dossier in dossiers]
    figures1,qss,i = [],[],0
    height = 85 if len(colors)==2\
        else 134 if len(colors)==3 else (154 if len(colors)==4 else 174)
    for one_dossier in dossiers: 
        i_col = 'alls. ' + str(one_dossier['q_in'][i]['c'])
        qs = [one['q'] for one in one_dossier['q_in']]
        x_names = [name.replace("Group","").replace("subm_","") for name in subm_names]
        width = 157  if len(colors) == 5\
            else (121 if len(colors) == 8\
            else (131 if len(colors) == 9\
            else (141 if len(colors) == 10\
            else (171 if len(colors) == 11 else 133))))
        f = figure(x_range=x_names,width=width, height=height, title=i_col)
        f.vbar(x=x_names, width=0.585, top=qs, color=colors)
        figures1.append(f)
        qss.append(qs)
        i+=1
    grid = gridplot([figures1])
    output_file('tida_alls.html')
    if show_figures1 == True: show(grid)
    sub_wts = params['subwts']
    main_wts = [subm['weight'] for subm in params['subm']]
    mms,acc_mass = [],[]
    for j in range(len(dossiers)):
        one_dossier = dossiers[j]
        qs = [one['q'] for one in one_dossier['q_in']]
        mm = [qs[h] * (main_wts[j] + sub_wts[h]) for h in range(len(sub_wts))]
        mass = sum(mm)
        mms.append(mm)
        acc_mass.append(round(mass))                        #subm_names[::-1]
    y_names = [name + " - " + str(mass) for name,mass in zip(subm_names,acc_mass)]
    f1 = figure(y_range=y_names, width=313, height=height, title='relations of general masses')
    f1.hbar(y=y_names, height=0.585, right=acc_mass, left=0, color=colors)
    output_file('tida_alls2.html')
    alls = [f'alls.{i}' for i in range(len(dossiers))]
    subm = [f'sub{i}'   for i in range(len(dossiers))] 
    mmsT  = np.asarray(mms).T
    data = {'cols' : alls}
    for i in range(len(dossiers)): data[f'sub{i}'] = mmsT[i,:]
    f2 = figure(y_range=alls, height=height, width=274, title=" ( relations of columns masses )")
    f2.hbar_stack(subm, y='cols', height=0.585, color=colors, source=data)
    qssT  = np.asarray(qss).T
    data = {'cols' : alls}
    for i in range(len(dossiers)): data[f'sub{i}'] = qssT[i,:]
    f3 = figure(y_range=alls, height=height, width=210, title="ratios in columns")
    f3.hbar_stack(subm, y='cols', height=0.585, color=colors, source=data)
    grid = gridplot([[f3,f2,f1]])
    show(grid)
    if show_figures2 == True:
        def read(params,i):
            FiN = params["path"] + params["subm"][i]["name"] + ".csv"
            target_name_back = {'target':params["target"],'pred':params["target"]}
            return pd.read_csv(FiN).rename(columns=target_name_back)
        dfs = [read(params,i) for i in range(len(params["subm"]))] + [df_cross]
        _height = 334 if len(params["subm"]) == 9 else 254
        f   = figure(width=785, height=_height)
        f.title.text = 'Click on legend entries to mute the corresponding lines'
        b,e        = 21000,21121
        line_x     = [dfs[i][b:e]['id']             for i in range(len(dfs))]
        line_y     = [dfs[i][b:e]['loan_paid_back'] for i in range(len(dfs))]
        color      = colors + [color_cross]
        alpha      = [0.8 for i in range(len(dfs)-1)] + [0.95]
        lws        = [1.0 for i in range(len(dfs)-1)] + [1.00]
        legend = subm_names + ['cross']
        for i in range(len(legend)):
            f.line(line_x[i], line_y[i], line_width=lws[i], color=color[i], alpha=alpha[i],
                   muted_color='white',legend_label=legend[i])
        f.legend.location = "top_left"
        f.legend.click_policy="mute"
        show(f)


def matrix_vs(path,fs_names):
    def load(path,fs_names):
        dfs = [pd.read_csv(path + name_subm +'.csv') for name_subm in fs_names]
        for i in range(len(dfs)):
            dfs[i] = dfs[i].rename(columns={"loan_paid_back": f'{fs_names[i]}'})
        dfsm = pd.merge(dfs[0], dfs[1], on="id")
        for i in range(2,len(dfs)):
            dfsm = pd.merge(dfsm,dfs[i],on='id')
        return dfsm   
    def make_list_vs(fs_names):
        list = []
        for i in range(0,len(fs_names)-1):
            for j in range(i+1,len(fs_names)):
                list.append(fs_names[i] + "_vs_" + fs_names[j])
        return list
    def get_mvs(dfs, list_vs):
        def get_abs_distance(x,t1,t2):
            return abs(x[t1]-x[t2])
        for vs in list_vs:
            t = vs.split('_vs_')
            dfs[vs] = dfs.apply(lambda x: get_abs_distance(x,t[0],t[1]), axis=1)
        return dfs   
    def distance_vs(name, st_names, list_vs, dfs):
        distances = []
        for st in st_names:
            vs_between = name + "_vs_" + st
            if vs_between not in list_vs:
                distances.append(0)
            else: distances.append(round(dfs[vs_between].sum()))
        return distances
    dfs = load(path,fs_names)
    list_vs = make_list_vs(fs_names)
    mvs = get_mvs(dfs, list_vs)
    m1 = pd.DataFrame({'subm':fs_names})
    m2 = pd.DataFrame({ name :distance_vs(name, fs_names, list_vs, mvs) for name in fs_names})
    matrix = pd.concat([m1,m2],axis=1)
    return matrix


def display_distances(params):
    files = [subm['name'] for subm in params['subm']]
    distances = matrix_vs ( params['path'], files )            
    display(distances)


def arr_colors(color):
    sg = ['silver','gainsboro']
    if color=='silver'or color=='S': return ['gray','silver','gold']          + sg
    if color=='red'   or color=='R': return ['darkorchid','orangered','crimson'] + sg
    if color=='green' or color=='G': return ['darkorchid','limegreen', 'darkgreen'] + sg
    if color=='blue'  or color=='B': return ['darkorchid','mediumblue','midnightblue'] + sg
    return ['black','dimgray','gray'] + sg


def convert(schema):
    colors = arr_colors(schema[2])
    dicts  = [
        {'name': schema[0][i],'weight':schema[1][i],'color':colors[i]} 
        for i in range(len(schema[0]))
    ]
    return {'subm':dicts}


def h_blend(
        params, _update={},
        cross='silver',
        details=False,
        fig1=False, fig2=False, wf2=555, 
        dtls=False, dist=False, subm=''):

    if isinstance(params, list): params = convert(params)

    if 'path' in _update: params.update(_update)
    
    color_cross, dk  = cross, copy.deepcopy(params)

    if details == True:
        dist = True
        show_details,show_figures1,show_figures2 = True,True,True
    else:
        show_details,show_figures1,show_figures2 = dtls,fig1,fig2
        
    file_short_names = [subm['name'] for subm in params['subm']]
    type_sort    = params['type_sort'][0]
    dk['asc']    = params['type_sort'][1]
    dk['desc']   = params['type_sort'][2]
    dk['id']     = params['id_target'][0]
    dk['target'] = params['id_target'][1]
# ------------------------------------------------------------------------
    def read(dk,i):
        tnm = dk["subm"][i]["name"]
        FiN = dk["path"] + tnm + ".csv"
        return pd.read_csv(FiN).rename(columns={
            'target':tnm, 'pred':tnm, dk["target"]:tnm})
        
    def merge(dfs_subm):
        df_subms = pd.merge(dfs_subm[0],  dfs_subm[1], on=[dk['id']])
        for i in range(2, len(dk["subm"])): 
            df_subms = pd.merge(df_subms, dfs_subm[i], on=[dk['id']])
        return df_subms
        
    def da(dk,sorting_direction,show_details):
        
        df_subms = merge([read(dk,i) for i in range(len(dk["subm"]))])
        cols = [col for col in df_subms.columns if col != dk['id']]
        short_name_cols = [c for c in cols]
        
        def alls1(x, sd=sorting_direction,cs=cols):
            reverse = True if sd=='desc' else False
            tes = {c: x[c] for c in cs}.items()
            subms_sorted = [t[0] for t in sorted(tes,key=lambda k:k[1],reverse=reverse)]
            return subms_sorted

        import random

        def alls2(x, sd=sorting_direction,cs=cols):
            reverse = True if sd=='desc' else False
            tes = {c: x[c] for c in cs}.items()
            subms_random = [t[0] for t in tes]
            random.shuffle(subms_random)
            return subms_random

        alls = alls1 if type_sort == 'asc/desc' else alls2
            
        def summa(x,cs,wts,ic_alls): 
            return sum([x[cs[j]] * (wts[0][j] + wts[1][ic_alls[j]]) for j in range(len(cs))])
            
        wts = [[[e['weight'] for e in dk["subm"]], [w for w in dk["subwts" ]]]]
          
        def correct(x, cs=cols, wts=wts):
            i = [x['alls'].index(c) for c in short_name_cols]
            return summa(x,cs,wts[0],i)

        if len(wts) == 1:
            correct_sub_weights = [wt for wt in dk["subwts"]]
            weights = [subm['weight'] for subm in dk["subm"]]
            def correct(x, cs=cols, w=weights, cw=correct_sub_weights):
                ic = [x['alls'].index(c) for c in short_name_cols]
                cS = [x[cols[j]] * (w[j] + cw[ic[j]]) for j in range(len(cols))]
                return sum(cS)
                   
        def amxm(x, cs=cols):
            list_values = x[cs].to_list()
            mxm = abs(max(list_values)-min(list_values))
            return mxm

        if len(wts) > 1:
            df_subms['mx-m']   = df_subms.apply(lambda x: amxm   (x), axis=1)
        df_subms['alls']       = df_subms.apply(lambda x: alls   (x), axis=1)
        df_subms[dk["target"]] = df_subms.apply(lambda x: correct(x), axis=1)
        schema_rename = { old_nc:new_shnc for old_nc, new_shnc in zip(cols, short_name_cols) }
        df_subms = df_subms.rename(columns=schema_rename)
        df_subms = df_subms.rename(columns={dk["target"]:"ensemble"})
        df_subms.insert(loc=1, column=' _ ', value=['   '] * len(df_subms))
        df_subms[' _ '] = df_subms[' _ '].astype(str)
        pd.set_option('display.max_rows',100)
        pd.set_option('display.float_format', '{:.5f}'.format)
        vcols = [dk['id']]+[' _ '] + short_name_cols + [' _ ']+['alls']+[' _ ']+['ensemble']
        if len(wts) > 1: vcols.append([' _ '] + ['mx-m'])
        df_subms = df_subms[vcols]
        if show_details and sorting_direction=='desc': display(df_subms.head(5))
        pd.set_option('display.float_format', '{:.5f}'.format)
        df_subms = df_subms.rename(columns={"ensemble":dk["target"]})
        if sorting_direction=='desc': 
            df_subms.to_csv(f'tida_{sorting_direction}.csv', index=False)
        return df_subms[[dk['id'],dk['target']]]
   
    def ensemble_da(dk,        show_details): 
        dfD    = da(dk,'desc', show_details)
        dfA    = da(dk,'asc',  show_details)
        dfA[dk['target']] = dk['desc']*dfD[dk['target']] + dfA[dk['target']]*dk['asc']
        return dfA

    da = ensemble_da(dk,show_details)
    bokeh_show(dk, da, show_figures1, show_figures2, wf2, color_cross)
    if dist == True: display_distances(params)
    if subm != '': da.to_csv(subm, index=False)
    return  da


def redirrect(full_FiN, short_FiN):
    df = pd.read_csv(full_FiN)
    df.to_csv(short_FiN, index=False)
    

def s_blend(df1,df2,df3, wts=[0.34, 0.30, 0.30], sub=''):
    t = 'loan_paid_back'
    df = pd.read_csv('/kaggle/input/playground-series-s5e11/sample_submission.csv')
    df[t] = df1[t]*wts[0] + df2[t]*wts[1] + df3[t]*wts[2]
    if sub != "":
        df.to_csv(sub, index=False)
        print(f'\n {sub} - ready to use \n')
    return df

def straight_blend(df1,df2,wts=[0.50,0.50]):
    df = pd.read_csv('/kaggle/input/playground-series-s5e11/sample_submission.csv')
    df[t] = df1[t]*wts[0] + df2[t]*wts[1]
    return df

def direct_blend(subm_file_1, subm_file_2, wts=[0.50,0.50]):
    df1 = pd.read_csv(subm_file_1)
    df2 = pd.read_csv(subm_file_2)
    return straight_blend(df1,df2,wts=wts)

Loading BokehJS ...

In [2]:
params = {
  'path'     : f"/kaggle/input/18-november-2025-ps-s5e11/submission 0.92767.",            
  'id_target': ['id',"loan_paid_back"],          
  'type_sort': ['asc/desc',0.33,0.67 ],
  'subwts'   : [ -0.008,-0.005,-0.004,-0.003, +0.002,+0.003,+0.004,+0.005,+0.006 ],
  'subm'     : [
      {'name': f'a', 'weight':+0.20, 'color':'gold'     },
      {'name': f'b', 'weight':+0.10, 'color':'orangered'  },
      {'name': f'c', 'weight':+0.10, 'color':'crimson'    },
      {'name': f'd', 'weight':+0.10, 'color':'limegreen'  },
      {'name': f'e', 'weight':+0.10, 'color':'forestgreen'},
      {'name': f'f', 'weight':+0.10, 'color':'darkgreen'  },
      {'name': f'g', 'weight':+0.10, 'color':'royalblue'  },
      {'name': f'h', 'weight':+0.10, 'color':'mediumblue' },
      {'name': f'i', 'weight':+0.10, 'color':'navy'       },]
}

df = h_blend(params, details=True, subm='pss5e11hb7g1.csv')

,id,_,a,b,c,d,e,f,g,h,i,_,alls,_,ensemble
0,593994,,0.94842,0.94865,0.94884,0.94893,0.94929,0.94902,0.94923,0.94915,0.94919,,"[e, g, i, h, f, d, c, b, a]",,0.94890
1,593995,,0.96820,0.96845,0.96866,0.96875,0.96913,0.96886,0.96915,0.96888,0.96902,,"[g, e, i, h, f, d, c, b, a]",,0.96872
2,593996,,0.43897,0.43865,0.43869,0.43870,0.43814,0.43872,0.43909,0.43711,0.43810,,"[g, a, f, d, c, b, e, i, h]",,0.43849
3,593997,,0.92249,0.92307,0.92360,0.92387,0.92476,0.92413,0.92468,0.92432,0.92450,,"[e, g, i, h, f, d, c, b, a]",,0.92376
4,593998,,0.96769,0.96792,0.96812,0.96821,0.96857,0.96830,0.96858,0.96834,0.96846,,"[g, e, i, h, f, d, c, b, a]",,0.96818


,subm,a,b,c,d,e,f,g,h,i
0,a,0,0,0,0,0,0,0,0,0
1,b,73,0,0,0,0,0,0,0,0
2,c,134,61,0,0,0,0,0,0,0
3,d,166,94,33,0,0,0,0,0,0
4,e,278,205,144,113,0,0,0,0,0
5,f,194,122,61,29,85,0,0,0,0
6,g,251,180,120,89,48,63,0,0,0
7,h,258,187,135,107,63,82,96,0,0
8,i,249,176,116,85,29,58,48,48,0


In [3]:
params = {
  'path'     : f"/kaggle/input/18-november-2025-ps-s5e11/submission 0.92767.",            
  'id_target': ['id',"loan_paid_back"],          
  'type_sort': ['asc/desc',0.33,0.67 ],
  'subwts'   : [ +0.02,+0.01,-0.005,-0.010,-0.015 ],
  'subm'     : [
      {'name': f'a', 'weight':+0.60, 'color':'gold'      },
      {'name': f'b', 'weight':+0.10, 'color':'maroon'    },
      {'name': f'c', 'weight':+0.10, 'color':'sienna'    },
      {'name': f'd', 'weight':+0.10, 'color':'chocolate' },
      {'name': f'e', 'weight':+0.10, 'color':'sandybrown'},]
}

df = h_blend(params, details=True, subm='pss5e11hb7g2.csv')

,id,_,a,b,c,d,e,_,alls,_,ensemble
0,593994,,0.94842,0.94865,0.94884,0.94893,0.94929,,"[e, d, c, b, a]",,0.94864
1,593995,,0.96820,0.96845,0.96866,0.96875,0.96913,,"[e, d, c, b, a]",,0.96844
2,593996,,0.43897,0.43865,0.43869,0.43870,0.43814,,"[a, d, c, b, e]",,0.43881
3,593997,,0.92249,0.92307,0.92360,0.92387,0.92476,,"[e, d, c, b, a]",,0.92307
4,593998,,0.96769,0.96792,0.96812,0.96821,0.96857,,"[e, d, c, b, a]",,0.96791


,subm,a,b,c,d,e
0,a,0,0,0,0,0
1,b,73,0,0,0,0
2,c,134,61,0,0,0
3,d,166,94,33,0,0
4,e,278,205,144,113,0


In [4]:
df.to_csv('submission.csv',index=False)
df

,id,loan_paid_back
0,593994,0.94863
1,593995,0.96843
2,593996,0.43880
3,593997,0.92304
4,593998,0.96790
...,...,...
254564,848558,0.97911
254565,848559,0.85755
254566,848560,0.94743
254567,848561,0.97558
